# Пелевин RNN - генерация текста

In [1]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn.functional as F

from torch.nn import Embedding, Linear, LSTM, Dropout

from IPython.display import clear_output

### Загрузка токенов

In [2]:
with open('tokens.txt') as t_file:
    tokens_string = t_file.read()
tokens = list(tokens_string)

In [3]:
token_to_id = {token: idx for idx, token in enumerate(tokens)}
num_tokens = len(tokens)

# print(tokens)

In [5]:
print(len(tokens))

214


### Загрузка вероятностей начала предложений (букв)

In [6]:
start_letters = []
start_probs = []

with open('start_of_sentences.txt') as s_file:
    for line in s_file:
        line = line.rstrip()
        letter, prob = line.split('_')
        start_letters.append(letter)
        start_probs.append(int(prob))

p_sum = sum(start_probs)
start_probs = [el/p_sum for el in start_probs]

# print(start_letters)
# print(p_sum)
# print(sum(start_probs))

In [7]:
def get_start_letter(start_letters, start_probs):
    letter = np.random.choice(start_letters, p=start_probs)
    return letter

# for i in range(1000):
#     print(get_start_letter(start_letters, start_probs), end='')

### Определение модели

In [8]:
class PelevinRNN(torch.nn.Module):
    def __init__(self, num_tokens=num_tokens, emb_size=100, rnn_num_units=512):
        super(self.__class__, self).__init__()
        self.emb = Embedding(num_tokens, emb_size)
        self.rnn = LSTM(emb_size, rnn_num_units, num_layers=3, batch_first=True)
        self.hid_to_logits = Linear(rnn_num_units, num_tokens)
        self.dropout = Dropout(p=0.6)

    def forward(self, x, hidden_state=None):
        x = self.dropout(self.emb(x))
        if hidden_state is not None:
            h_seq, new_hidden = self.rnn(x, hidden_state)
        else:
            h_seq, new_hidden = self.rnn(x)
        next_logits = self.hid_to_logits(h_seq)
        next_logp = F.log_softmax(next_logits, dim=-1)
        return next_logp, new_hidden

In [9]:
model = PelevinRNN().cuda()
model.eval()

### Определение функции генерации семплов

In [10]:
MAX_LEN = 800

def generate_sample(model, seed_phrase=None, max_length=MAX_LEN, temperature=1.0):

    if seed_phrase is None:
        seed_phrase = get_start_letter(start_letters, start_probs)

    if seed_phrase == '':
        return ''
    
    seq = [[token_to_id[token] for token in seed_phrase]]
    seq = torch.tensor([seq], dtype=torch.int64).cuda()
    
    hidden_s = None
    for i in range(len(seed_phrase) - 1):
        _, hidden_s = model.forward(seq[:, :, i], hidden_s)
    
    for _ in range(max_length - len(seed_phrase)):
        logp_next, hidden_s = model.forward(seq[:, :, -1], hidden_s)
        p_next = F.softmax(logp_next / temperature, dim=-1).data.cpu().numpy()[0]
        next_ix = np.random.choice(len(tokens), p=p_next[0])
        next_ix = torch.tensor([[[next_ix]]], dtype=torch.int64).cuda()
        seq = torch.cat([seq, next_ix], dim=2)
        
    return ''.join([tokens[ix] for ix in seq[0, 0].data.cpu().numpy()])

### Загрузка весов модели

In [11]:
model.load_state_dict(torch.load('model.lol'))

<All keys matched successfully>

In [12]:
model.parameters

<bound method Module.parameters of PelevinRNN(
  (emb): Embedding(214, 100)
  (rnn): LSTM(100, 512, num_layers=3, batch_first=True)
  (hid_to_logits): Linear(in_features=512, out_features=214, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
)>

### Веселимся!

In [15]:
for _ in range(30):
    sample = generate_sample(model, seed_phrase=None, temperature=0.4)
    print(sample)

— А что тебе не просто получается?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
По этому пространство прикрепились в сторону с киной и стояли две пальцев в капиталину по коридору.                                                                                                    

In [32]:
start_symbol = 'Однажды'
for _ in range(10):
    sample = generate_sample(model, seed_phrase=start_symbol, temperature=0.5)
    print(sample)

Однажды в своем причине, когда была открыта только такое положеное в массовом полосе – и была сделана с собой по небесному волку, и полностью открылась.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
Однажды он просвечивал на их пола и после этого он повернулся к половину рукой по водой и поставил полной полосе в полу волосы деревья и положила руку с собой в сторону с собой ладонью в половине.   